In [58]:
import openai, json
from openai.types.chat import ChatCompletionMessage

client = openai.OpenAI()
messages = []


In [59]:
def get_weather(city):
    return '15 degrees Celsius.'


FUNCTION_MAP = {
    "get_weather": get_weather
}

In [60]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "A function to get the weather of a city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the weather of.",
                    },
                },
                "required": ["city"],
            }
        }
    }
]

In [61]:
def process_ai_response(message: ChatCompletionMessage):

    if message.tool_calls:
        messages.append({
            "role": "assistant",
            "content": message.content or "",
            "tool_calls": [
                {
                    "id": tool_call.id,
                    "type": "function",
                    "function": {
                        "name": tool_call.function.name,
                        "arguments": tool_call.function.arguments
                    }
                } for tool_call in message.tool_calls]
        })

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            print(f"Calling function: {function_name} with arguments: {arguments}")

            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}

            function_to_run = FUNCTION_MAP.get(function_name)

            result = function_to_run(**arguments)

            print(f"Ran function: {function_name} with arguments: {arguments} and result: {result}")

            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": function_name,
                "content": result
            })

        call_ai()
    else:
        messages.append({"role": "assistant", "content": message.content})
        print(f"AI: {message.content}")


In [62]:
def call_ai():
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=messages,
        tools=TOOLS
    )
    process_ai_response(response.choices[0].message)

In [63]:
while True:
    message = input("Send a message to the LLM...")
    if message == 'quit' or message == 'q':
        break
    else:
        messages.append({"role": "user", "content": message})
        print(f"User: {message}")
        call_ai()


User: My name is Chaehyeon
AI: Nice to meet you, Chaehyeon! How can I help you today? If you’d like, I’ll use your name in our chats. What would you like to do or talk about?
User: What is my name?
AI: Your name is Chaehyeon. I can use it in our chats if you’d like. Would you like me to address you as Chaehyeon from now on?
User: What is the weather in Seoul?
Calling function: get_weather with arguments: {"city":"Seoul"}
Ran function: get_weather with arguments: {'city': 'Seoul'} and result: 15 degrees Celsius.
AI: Chaehyeon, the current temperature in Seoul is 15°C. Would you like an hourly forecast or more details (e.g., rain chances)?


In [64]:
messages

[{'role': 'user', 'content': 'My name is Chaehyeon'},
 {'role': 'assistant',
  'content': 'Nice to meet you, Chaehyeon! How can I help you today? If you’d like, I’ll use your name in our chats. What would you like to do or talk about?'},
 {'role': 'user', 'content': 'What is my name?'},
 {'role': 'assistant',
  'content': 'Your name is Chaehyeon. I can use it in our chats if you’d like. Would you like me to address you as Chaehyeon from now on?'},
 {'role': 'user', 'content': 'What is the weather in Seoul?'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'id': 'call_m4kzxZOWvelD5Mc5iVUifI8C',
    'type': 'function',
    'function': {'name': 'get_weather', 'arguments': '{"city":"Seoul"}'}}]},
 {'role': 'tool',
  'tool_call_id': 'call_m4kzxZOWvelD5Mc5iVUifI8C',
  'name': 'get_weather',
  'content': '15 degrees Celsius.'},
 {'role': 'assistant',
  'content': 'Chaehyeon, the current temperature in Seoul is 15°C. Would you like an hourly forecast or more details (e.g., rain chan